In [1]:
from typing import Dict, List, KeysView
from dataclasses import dataclass
from gurobipy import Model, GRB

In [2]:
class InvalidKnapsackError(RuntimeError):
    pass

class Knapsack:
    profit: Dict[int, int]
    weight: Dict[int, int]
    capacity: int

    def __init__(self, profit: Dict[int, int], weight: Dict[int, int], capacity: int):
        if set(profit.keys()) != set(weight.keys()):
            raise InvalidKnapsackError("Profit and weights must have the same keys.")
        
        if capacity < 0:
            raise InvalidKnapsackError("Capaity cannot be negative")

        self.profit = profit
        self.weight = weight
        self.capacity = capacity

    def items(self) -> KeysView:
        return self.weight.keys()


@dataclass
class KnapsackSolution:
    profit: int
    items: List[int]

In [3]:
weight = {j: w for j, w in enumerate((2,2,2,4,4,6))}
profit = {j: p for j, p in enumerate((5,4,3,5,4,5))}
kp = Knapsack(weight=weight, profit=profit, capacity=12)

In [4]:
def solve_mip(kp: Knapsack) -> KnapsackSolution:
    m = Model()
    x = m.addVars(kp.items(), vtype=GRB.BINARY, obj=kp.profit, name='x')
    m.addConstr(
        (sum(kp.weight[j] * x[j] for j in kp.items()) <= kp.capacity),
        name='capacity')
    m.ModelSense = GRB.MAXIMIZE

    m.optimize()
    
    assert m.Status == GRB.OPTIMAL

    items = [j for j in kp.items() if x[j].X > 0.5]
    profit = sum(kp.profit[j] for j in items)

    return KnapsackSolution(profit=profit, items=items)

In [5]:
solve_mip(kp=kp)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2555908
Academic license 2555908 - for non-commercial use only - registered to al___@upf.edu
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2555908 - for non-commercial use only - registered to al___@upf.edu
Optimize a model with 1 rows, 6 columns and 6 nonzeros
Model fingerprint: 0x70c446c8
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [2e+00, 6e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]
Found heuristic solution: objective 17.0000000
Presolve removed 1 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.06 seconds (0.00 work units)
T

Best objective 1.800000000000e+01, best bound 1.800000000000e+01, gap 0.0000%


KnapsackSolution(profit=18, items=[0, 1, 3, 4])